# Libraries

In [23]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from sklearn import metrics

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Import data

In [24]:
%run Data_preprocessing.ipynb

************************************************************
There are 6 set of X
X_*_nozero
------------------------------------------------------------
Normalized version
X_*_norm
------------------------------------------------------------
There are 4 set of Y
y_train_t1, y_train_t2, y_valid_t1, y_valid_t2
when training, please use: 'y_train_t1_value,y_train_t2_value,y_valid_t1_value,y_valid_t2_value'
************************************************************


In [25]:
print(X_train_norm.shape)
print(X_valid_norm.shape)
print(X_test_norm.shape)

(16760, 61)
(2394, 61)
(4790, 61)


# Imbalanced Learning

In [26]:
Full = pd.DataFrame(np.concatenate((X_train_norm,pd.DataFrame(y_train_t1.iloc[:,-1])),axis=1))
Full

label0 = Full[Full[61]==0]
label1 = Full[Full[61]==1]
print(label0.shape,label1.shape)

(15535, 62) (1225, 62)


In [27]:
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler()
x_over, y_over = oversample.fit_resample(X_train_norm, y_train_t1.iloc[:,-1])


y_over = pd.DataFrame(y_over)
y_over.shape
x_over.shape

(31070, 61)

In [28]:
Full = pd.DataFrame(np.concatenate((x_over,y_over),axis=1))
Full

label0 = Full[Full[61]==0]
label1 = Full[Full[61]==1]
print(label0.shape,label1.shape)

(15535, 62) (15535, 62)


In [33]:
X_train_norm=x_over
y_train_t1 = y_over
y_train_t1

,mort_icu
0,0
1,0
2,1
3,0
4,0
...,...
31065,1
31066,1
31067,1
31068,1


In [34]:
Full_train = pd.DataFrame(np.concatenate((X_train_norm,pd.DataFrame(y_train_t1.iloc[:,-1])),axis=1))
Full_train

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,-0.288458,-3.306524e+00,-0.448675,0.480040,-0.220009,0.000000,-1.643837,-3.233569e-01,0.936056,-1.378320,...,0.064360,-0.867032,0.884117,5.371056e-01,6.239997e-01,0.000000,4.104534e-17,1.244821,0.790869,0.0
1,-0.293777,-4.706776e-01,-0.635565,1.973117,-0.292865,-1.171347,-1.405521,-5.765104e-01,2.121048,-0.351525,...,-1.068142,0.309697,-0.102469,3.751961e-17,-8.842220e-18,0.000000,4.104534e-17,0.000000,-0.173856,0.0
2,-0.325687,1.138720e-17,-0.535891,-0.117191,-0.338660,3.287565,0.501009,-4.921259e-01,-0.351978,0.760836,...,-0.615141,2.663156,0.822486,1.703934e+00,1.692524e+00,0.000000,4.104534e-17,0.000000,0.190857,1.0
3,0.000000,1.138720e-17,0.000000,0.181425,0.000000,0.000000,0.501009,2.398447e-17,-0.248935,1.103101,...,0.743862,-0.537548,-0.719132,3.751961e-17,-8.842220e-18,0.000000,4.104534e-17,0.000000,0.084973,0.0
4,0.000000,1.138720e-17,0.000000,-0.714421,0.000000,0.000000,0.262693,2.398447e-17,0.781492,0.504137,...,1.423363,1.015735,0.514121,5.954471e-01,6.239997e-01,-2.302771,-5.419998e-01,0.000000,-0.503274,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31065,0.000000,1.138720e-17,0.000000,-1.311652,0.000000,0.000000,1.930907,2.398447e-17,0.060193,0.675270,...,0.517361,-1.526001,-0.642023,-4.352512e-01,-8.842220e-18,0.000000,4.104534e-17,-0.697487,-0.162091,1.0
31066,-0.325687,-4.251806e+00,-0.211947,-1.908882,-0.322007,-0.765991,0.024377,-5.343181e-01,-0.609585,-0.522657,...,0.517361,-1.102378,-0.487867,3.751961e-17,-8.842220e-18,0.000000,4.104534e-17,-1.879182,3.708572,1.0
31067,-0.283140,-2.361242e+00,-0.236866,0.480040,-0.276212,-1.171347,-1.643837,-4.499337e-01,2.172569,-1.036055,...,0.290861,0.121421,-0.642023,-1.115901e+00,-1.299345e+00,0.000000,4.104534e-17,-1.033814,1.061462,1.0
31068,-0.229956,-1.652280e+00,-0.411297,-0.415806,-0.324089,0.000000,1.215958,-4.499337e-01,0.884535,-0.608224,...,0.743862,-1.008240,-0.719101,4.120368e-02,6.239997e-01,0.171268,4.104534e-17,0.653022,3.402684,1.0


In [35]:
Full_valid = pd.DataFrame(np.concatenate((X_valid_norm,pd.DataFrame(y_valid_t1.iloc[:,-1])),axis=1))
Full_valid

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,-2.666190e-01,-9.313850e-01,11.130149,0.542448,-2.140465e-01,-1.389642e+00,-0.711125,-3.701721e-02,-0.691308,-0.834469,...,0.501611,-0.035861,-0.458333,-1.772923e-17,0.000000,1.152006e-17,1.287366e-17,-1.474159,-0.342432,0.0
1,-5.576500e-02,2.914430e-18,-0.532450,4.214881,1.213492e-01,1.348945e-17,-2.143089,-4.881871e-01,3.879406,0.407855,...,-0.182965,-0.333034,-1.070428,-1.772923e-17,0.000000,1.152006e-17,1.770704e+00,-0.854172,1.455213,0.0
2,-2.687489e-01,3.635407e-02,-1.309956,-1.905841,-2.044638e-01,-8.878269e-01,0.004857,-4.881871e-01,0.501052,-0.420361,...,0.045227,-0.382563,-0.305325,-1.772923e-17,0.000000,1.152006e-17,1.763050e-01,0.943791,-0.042825,0.0
3,2.763795e-17,2.914430e-18,0.000000,0.000000,-2.803262e-17,1.348945e-17,0.482179,2.296752e-17,-0.343536,0.000000,...,-0.182965,-0.481621,1.836832,2.146152e+00,2.654380,1.152006e-17,1.287366e-17,1.026456,0.950611,0.0
4,-2.623594e-01,-2.055807e-01,-0.654412,-0.375661,-2.827228e-01,-3.860117e-01,0.004857,-4.477838e-01,-0.890035,-1.662684,...,1.186187,-0.877852,0.230230,7.677421e-01,0.670101,7.360628e-01,1.287366e-17,-0.523512,-0.358201,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2389,-2.155029e-01,2.914430e-18,-0.944071,-1.293769,-1.916868e-01,1.348945e-17,1.675483,-4.881871e-01,-0.939716,-0.972504,...,-0.411156,0.409898,-0.458355,-1.772923e-17,0.000000,1.152006e-17,1.287366e-17,-0.995327,1.565595,0.0
2390,-2.943069e-01,4.391180e+00,-1.111768,-1.599805,-2.939026e-01,1.158035e-01,-1.427107,-5.285903e-01,-0.939716,-0.282325,...,-0.411156,0.409898,-1.070410,-1.772923e-17,0.000000,1.152006e-17,1.287366e-17,-1.575593,0.067557,0.0
2391,-2.921771e-01,-6.894502e-01,1.632373,1.154520,-2.715429e-01,1.348945e-17,0.004857,-5.285903e-01,0.848824,-0.006253,...,-2.464883,-0.630207,-0.840889,-1.106896e+00,-1.314179,-2.074230e+00,-6.580695e-01,-1.098442,0.225245,1.0
2392,2.763795e-17,2.914430e-18,0.000000,-0.375661,-2.803262e-17,1.348945e-17,2.868786,2.296752e-17,1.345641,1.098035,...,-1.780307,-0.976909,-0.993903,-1.772923e-17,0.000000,1.152006e-17,1.287366e-17,0.349636,-0.405508,0.0


In [39]:
num_col_names = Full_train.columns[:-1]
num_col_names

RangeIndex(start=0, stop=61, step=1)

In [43]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig


data_config = DataConfig(
    target=[61], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=[],
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=100,
    gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()

model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="60-30-15",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

tabular_model.fit(train=Full_train, validation=Full_valid)
result = tabular_model.evaluate(test)
pred_df = tabular_model.predict(test)


ImportError: cannot import name 'get_num_classes' from 'torchmetrics.utilities.data' (/home/fish/anaconda3/envs/comp3340/lib/python3.7/site-packages/torchmetrics/utilities/data.py)